In [6]:
import torchvision.models as models
from torchvision import datasets, transforms
import torch.nn as nn
import torch

In [7]:

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载数据集
dataset = datasets.ImageFolder(root='pathogen', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [3]:
print(dataset.classes)

['bacteria', 'fungi', 'healthy', 'pests', 'virus']


In [4]:
model = models.resnet50(pretrained=True)

# 冻结所有参数，以防在训练初期被破坏
for param in model.parameters():
    param.requires_grad = False

# 替换最后的全连接层，以匹配你的数据集类别数
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

d:\code\miniconda\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
d:\code\miniconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# model.to('cuda')  # 如果你有GPU的话

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Training Loss: {epoch_loss:.4f}')

# 训练模型
for epoch in range(5):  # 训练5个epochs
    train_epoch(model, dataloader, criterion, optimizer, 'cpu')

Training Loss: 0.6444
Training Loss: 0.2340
Training Loss: 0.1601
Training Loss: 0.1222
Training Loss: 0.0959


In [9]:
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
    avg_loss = total_loss / len(dataloader.dataset)
    accuracy = correct.double() / len(dataloader.dataset)
    print(f'Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

# 评估模型
evaluate_model(model, dataloader, criterion)

Loss: 0.0572, Accuracy: 0.9905


In [10]:
torch.save(model, 'model_recognition.pth')